### <center>Qu Capital - Onboarding Project by Thomas Rosal</center>
---

### Contents
1. [Setup](#setup)
2. [Summary](#summary)
3. [Literature Review](#lit)
4. [Underlying Economics](#econ)
6. [Potential for Confounding Variables](#con)
7. [Data](#data)
8. [Exploring Correlations](#cor)
9. [Trading Strategy Thesis](#strat)
9. [Trading Strategy Implementation - Naive Strategy](#naive)
11. [Trading Strategy Implementation - Improved Strategy](#improved)
12. [Results from Out-of-Sample Validation Backtests](#backtest)
13. [Discussion and Further Research](#disc)
14. [Bibliography](#bib)

---
### Setup <a name="setup"></a>
To execute this notebook requires the following, with relevant modules/libraries and data attached in the provided zip file:
1. Pyalgotrade implementation for Python 3
2. Modified Pyalgotrade libraries bar.py and plotter.py 
3. QuProject.py module, where the bulk of code (classes, functions, data munging) for this project resides
4. matplotlib, pandas, numpy Python modules
5. Data for AUD/USD, GBP/USD, EUR/USD, Big Mac Index 
6. (Optional) QuProject.R and the rmgarch library for R to implement DCC models
 
Instructions:
1. If you have not, install the libraries mentioned above
2. Unzip the attachment with this notebook in your current working directory, as the code relies on specifically defined file paths (especially the training and test data in the QuTrain and QuTest folders, respectively)
3. Replace the modified PyAlgoTrade modules (bar.py and plotter.py) in () with the modified versions provided
4. Execute the notebook!
 
Notes:
1. For techniques such as dynamic conditional correlation and some exploratory analysis, R was used (in part because a DCC implementation does not yet exist for Python); where relevant, the R code is also included.   

---
### Summary <a name="summary"></a>
The goal of this project is to explore the relationship between equity volatility and currency markets and exploiting this with a trading strategy. Specifically, we investigate the AUD/USD exchange rate and the VIX "fear gauge" equity volatility index. After a review of relevant literature, we present the underlying economic rationale behind price movements in the US dollar, the VIX, and potential correlations between the two. Building on these insights, we develop an investment/strategy thesis to exploit  signals from the relationship between the Dollar and the VIX.   

---
### Literature Review <a name="lit"></a>
There is a host of academic literature around correlations between the prices of securities, given its immense relevance in asset pricing and portfolio construction. Most studies typically focus on cross sectional relationships between securities and temporal relationships between similar type securities (pairs of currencies, various stock indices, etc.), but none so far have directly investigated the relationship between exchange rates and equity volatility. However, there are several studies, the most relevant of which discussed below, whose methods and insights contribute to our line of inquiry. Generally, these studies deal with the relationship between equity indices and currencies, various models that account for heteroskedasticity in financial time series data, and other relevant econometric and statistical specifications. 

The earliest of the relevant studies [[1]](https://www.bis.org/publ/confp01l.pdf) investigates if implied volatilities from currency-based options inform exchange rate movements; specifically, the study considers risk reverals, derivatives whose payoff can turn positive or negative contingent on large deviations of currency spot rates to their forward rates, and hence are considered the market's best guess as to the directional bias of the currency. A stochastic volatility approach is used to show that changes in the prices of risk reverals capture the comovements or the directional bias of currency spot rates and their volatility.    

In an alternative approach, [[2]](https://arxiv.org/abs/1002.0321) considers the cross-correlations in the S&P500 and Dow Jones Euro Stoxx 50 through an eigvenvalue spectrum-RMT approach, investigating over sliding windows the spectrum of the correlation matrices of the constituent equities of both indices to see if there is some additional, uncaptured correlation structure in the random part of the eigenvalue spectrum (as defined in RMT). Among its results, the study shows that drawdowns on the indices were associated with periods when the largest eigenvalue was greatest and positive returns with periods when the largest eigenvalue was smallest.    

More recent literature, however, have been using multivariate GARCH approaches in modelling correlations between asset returns, motivated largely by the heteroskedasticity endemic in financial data. The earliest of which, [[3]](http://www.nber.org/papers/w5974), considers a very similar problem - assessing the accuracy of correlations implied volatilities from currency options in forecasting exchange rate movements - to [[1]](https://www.bis.org/publ/confp01l.pdf), but employs a GARCH approach while comparing its results to historical correlations and JP Morgan RiskMetrics' exponential weighted moving average correlation, finding that, as in the previous study, implied volatilities yield the best forecasts and, hence, contain relevant information with regards to their respective exchange rates. Several studies also use GARCH models to investigate the relationship between trading volumes and asset returns, typically finding that GARCH models succeed where conventional ones do not. In particular, [[4]](http://www.greta.it/wp/03.08.PDF) uses a GJR-GARCH model to uncover statistically significant relationships between the S&P100 and the Nikkei 225, while [[5]](https://www.sciencedirect.com/science/article/pii/S1044028312000026) finds causality (and its direction) between return volatility, trading volume, and stock returns in emerging Asian markets from analysis of a series of bivariate GARCH models. 

Building on the multivariate GARCH model, some studies have also examined the Dynamic Condtional Correlation model, a specific class of the GARCH models, and its use in the analysis and prediction of financial time series. Crucially, DCC models enable the tracing of the evolution of correlations (conditional correlations) between pairs of assets through time, hence the term "dynamic", while keeping estimated models parsimonious. Several studies have found varying degrees of success in its use. [[6]](https://www.sciencedirect.com/science/article/pii/S1049007807000590) uses DCC models to examine the dynamics between financial markets in Thailand and in Indonesia, finding that tight monetary policy by Thailand had little effect on Indonesia towards the end of 1998, and that the statistically significant correlations reveal some interdependence between both countries, which is, considerably lower in a crisis. [[7]](http://www.ccsenet.org/journal/index.php/ijef/article/view/5922) finds contrary results in an examination of the financial contagion following the 07-08 crisis; in examining the stock indices from 5 developed and 10 emerging countries, the study shows that the correlations between these countries considerably increased following the start of the financial crisis. Indeed, using the conditional correlations, the study is able to divide the countries into 3 groups of varying exposure to the US during the crisis. Other studies that use DCC models include [[8]](https://www.tandfonline.com/doi/abs/10.1080/13504850500447331),[[9]](https://www.tandfonline.com/doi/abs/10.1080/17446540500428843),[[10]](https://www.sciencedirect.com/science/article/pii/S030142151500004X). 

There is also a subset of literature that deals specifically with cointegrated financial time series, typically asset returns on some pair or group of assets with mean-reverting behavior relative to each other. This research is aimed particularly at exploting this behavior through pairs trading, a form of arbitrage, popularly employed in forex trading of pairs of currencies. [[11]](https://www1.unisg.ch/www/edis.nsf/SysLkpByIdentifier/4600/$FILE/dis4600.pdf) delivers an exposition around the subject, providing examples for how this is employed. 

Building on this literature, our investigation of the correlations between equity volatility and exchange rates incoporate the insights from these studies, particularly those of heteroskedasticity in dealing with time series data. From this, we explore correlations in the usual way. We also check if our series are cointegrated to see if pair trading strategies are feasible. Then further building on the use of multivariate GARCH models in the literature, we use a Dynamic Conditional Correlation model to examine the US dollar (AUD/USD exchange rate) and equity volatility (VIX), recovering conditional correlations through time in the process. The goal is to incorporate these insights and analytics to improve upon the results of a naive USD-long bias trading strategy.   

---
### Underlying Economics <a name="econ"></a>
In this project, we consider two assets: the US Dollar, defined in relation to the Australian Dollar, and the VIX, the plots of which from July 2012 to July 2016 are shown; log returns are underneath. 

<img src="img/USD-VIX.png">

#### US Dollar
The value of a currency relative to others is fundamentally a reflection of supply and demand of that currency; generally, if investors or the market prefer one currency over another, then the price of the former rises relative to latter. Several factors are known to affect the demand and supply of currencies, thereby impacting their exchange rates on either a short and/or long term horizon. These include the following, with a brief description of the dynamics:

- **Trade balance of underwriting government**: countries with large deficits imply that forex earnings are less than forex expenditures; demand for foreign currencies increase, thereby increasing the exchange rate, depreciating the currency of the high-defecit country.
- **Interest rate levels**: when a country has a higher domestic interest rate that foreign rates, capital inflow will occur (b/c higher return for investors) and increase demand for the domestic currency, thereby causing it to appreciate. 
- **Inflation**: when the inflation of a country rises, its currency's purchasing power declines, so the domestic currency depreciates relative to foreign currencies with relatively lower inflation.
- **Government intervention (fiscal and monetary policy)**: in addition to setting interest rates, governments through monetary policy, such as open market interventions, or even through massive expenditures, impact domestic currency valuations; for example, expansionary monetary and fiscal policy will tend to cause expediture deficits, which in turn devalue the domestic currency; anohter example is direct purchases of foreign currencies in large quantities, which many governments maintain; the exchange of such large quantities directly impact supply and demand, and hence, exchange rates.  
- **Economic strength**: strong/relatively more valuable domestic currencies **can** potentially be a sign of a country's economic strength, but this has to be considered with the other mentioned variables; overvalued exchange rates, as in the case of Greece in their recent quagmire, can cause lower economic growth.  
- **Safety**: in turbulent markets, some currencies of countries known for stability tend to appreciate, as thdid the Swiss Franc in 2012 relative to the rest of Euro currencies. 

In our investigation, we consider primarily the exchange rate between the Australian Dollar and the US Dollar. 

#### VIX Index
The CBOE Volatility Index (VIX), popularly known as the "fear gauge" is a measure of the market's 30-day expectation of volatility in the S&P500. Quoted in percentage points, the VIX Index, an implied volatility, is a weighted average of various put and call options on the S&P500 Index. Higher VIX Index values correspond to riskier market expectations. 

As a common measure of market volatility, the VIX is typically compared to the S&P500 in literature. The CBOE asserts, on its [website](http://www.cboe.com/products/vix-index-volatility/vix-options-and-futures/vix-index/the-relationship-of-the-spx-and-the-vix-index) and in a white paper [[12]](https://www.cboe.com/micro/vix/vixwhite.pdf), the VIX's inverse relationship with the S&P500, but opinions diverge in academia. In particular, [[13]](https://onlinelibrary.wiley.com/doi/pdf/10.1002/fut.20532) studies the information content in VIX options, which enabling trading the VIX Index (which itself is not technically tradable) in comparison to S&P500 options, as they theoretically ought to contain the same information; unfortunately, it finds that the information content of the volatilities implied by these two classes of options diverge, although improvements on forecasts of S&P500 direction can be improved incorporating implied volatilities from both S&P500 and VIX options.  

---
### Potential for Confounding Variables <a name="con"></a>
Given the earlier discussion on the factors that affect the valuation of a currency, it is entirely possible that our trading strategy does not control for these unknowns. As a motivating example, consider the low interest rate regime in the US following the financial crisis; such has depressed dollar valuations globally, but this is not necessarily a factor that is captured by variations in the VIX Index or the relative under/over valuation of the dollar. As another example, consider the most recent Davos conference earlier this year, when Secretary of the Treasury Steve Mnuchin's [assertion](https://www.cnbc.com/2018/01/24/dollar-tanks-the-most-in-10-months-after-mnuchin-says-weak-dollar-is-good.html) that "Obviously a weaker dollar is good for us as it relates to trade and opportunities" caused the US Dollar to plummet to a nearly 1-year low.  It can be argued that potentially confounding variables and other events such as those mentioned may impact any correlations or other relationships in our statistical analyses, models, and ultimately, our trading strategies. 

---
### Data <a name="data"></a>
As mentioned already mentioned earlier, we use the AUD/USD spot rate, from the Reserve Bank of Australia, as a proxy for the US dollar, and the VIX Index, from the CBOE, as a proxy for equity volatility in the US market. We use 6 years' worth of daily data, amounting to prices for 1,469 days, from June 29, 2012 to June 29, 2018.  

In addition to these pre-specified data, we also use the following:
1. **GBR/USD** and **EUR/USD** daily spot rates from the Bank of England and the St. Louis Federal Reserve, respectively, since popular opinion among analysts suggests that these currency pairs tend to move together. These are also used to later backtest our improved trading strategy. Notably, the data from the St. Louis Fed feature NA values for some dates. In this case, we linearly interpolate between them.  
2. **Big Mac Index**, measured twice a year, by and from the Economist. We use data from 2012 to 2018 for Australia, Britain, and the Euro area. 

For training and testing purposes, we divide our data into two three year periods: June 29, 2012 to June 30, 2015 (735 data points), and then from July 1, 2015 to June 29, 2018 (734 data points). These are our training and testing sets, respectively. Backtests and validation are conducted on the test set. Furthermore, the training and test sets are also further divided into 46 rolling windows each (a total of 92) 486 day (approx. 2 years), from which summary statistics of relevant strategy performance metrics are collected and reported.    

---
### Exploring Correlations <a name="cor"></a> 

We would like to see if there is a (potentiall statistically) significant enough relationship between the VIX and USD exchange rates. The aim of this exercise is to see if we can incorporate some statistical measure of correlation, if it exists, in a trading strategy.  

To avoid spurious correlations computed from price levels, we compute the correlations for the first differenced and the log returns for USD and VIX series. Below are scatterplots of both:

<img src="img/Scatter-2.png">

By inspection there seems to be no correlation; Pearson's correlation coefficient for the first differenced and log return pairs are -0.0329 and -0.0358, respectively. This result is hardly surprising, as daily financial data is known to be extremely noisy. Instead, it may be possible to find meaningful correlations by expanding our time horizin and considering weekly and weekly data. The scatterplots for weekly and monthly first differenced series and log returns are below: 

<img src="img/weekmonth.png">

While the weekly and monthly scatterplots look just as discouraging, we find improvements their correlations:

|                   | Weekly  |Monthly |
| --- |:---:|:----:|
| First Differenced | -0.0827 | 0.1206 |
| Log Returns       | -0.0743 | 0.0877 |

These weekly and monthly correlations, while small, suggest the existence, at least, of some relationship between the pair. This suggests that one improvement over the naive strategy can involve incorporating a rolling mean for the VIX with a 1-week to 1-month (as opposed to a historical, static mean over the entire period) can lead to better trading signals. 

However, a drawback of the correlations computed above is that they are static throughout the window. Many studies in the literature, as well as empirical verifications, have shown that financial time series data tend to express 'volatility clustering' in that times of volatility tend to happen together. As an example, [[14]](https://www.cfapubs.org/doi/pdf/10.2469/cp.v26.n1.9) shows that the returns on US and foreign equities have a correlation of -17% when both markets' returns are one standard deviation above their means, but display a correlation of +76% when both markets' returns are one standard deviation below the mean. Hence, if we are to incorporate correlations into our trading strategy, we must aim to compute some correlation that incorporates the temporal component of our time series data. 

First, we look at the cross-correlation between both series. Before we do this, we perform checks for stationarity as a prerequisite for the CCF. The following are the p-values from ADF and KPSS tests for stationarity on the daily log returns and first differences:

|                   | ADF  |KPSS |
| --- |:---:|:----:|
| First Differenced | 0.01 | 0.1 |
| Log Returns       | 0.01 | 0.1 |

The < 0.05 p-value for the ADF and the > 0.05 p-value of the KPSS test convince us that the pairs of transformed series are, indeed, stationary. Examination of their ACFs, shown below for the log returns (since first differenced series is nearly identical), further confirm this, as the lags die out quickly: 

<img src="img/ACFs.png">

Now, we check their CCF: 

<img src="img/CCF.png">

Notice that we have significant positive correlations at lags -10, 3, and 6, and significant negative correlations at lag 14 and 18. These suggest that the VIX at time $t$ is positively correlated with AUD/USD spot rates for times $t-10$, $t+3$, and $t+6$, while it is negatively correlated with the AUD/USD for times $t+14$ and $t+18$. (Is there an economic interpretation here? Unclear.)

Now, we note that from here onwards, we will be using the pair of log return series as opposed to the first differenced pair, due largely in part to the nice interpretation of log returns (i.e. additive, so can compound growth). 

While the CCF did not necessarily lead to any meaningful economic interpretation, we can now look towards estimating a Dynamic Conditional Correlation model to the log returns of the AUD/USD and the VIX to generate conditional correlations through time for investigation. Unfortunately, Python does not have an implementation for DCC models, but R, through the library ```rmgarch```, we fit a DCC-GARCH model to the daily log returns of our data. 
We plot the conditional correlations from the DCC fit, with +1/-1 as the red dashed lines, and below it, the AUD/USD spot rate:

<img src="img/dccfit.png">

Initial inspections of these correlations in relation to the AUD/USD spot rate reveal potential sell signals at the two highest peaks in the first plot, around April-May 2013 and in October 2013; notably, these also corresponded to declines in the S&P500 Index during that year. However, downward movements in the conditional correlations seem inconclusive. All together, these seem to suggest high correlations during drawdown periods, while inconclusive for drawups. Also, note that the mean of the time varying correlations correspond to the correlations of the daily log returns of the AUD/USD spot price and the VIX.  

Placed here is the R code to fit the DCC model and generate the correlation series shown above. A clear (the best I could find) exposition on the math behind the DCC can be found [here](https://daim.idi.ntnu.no/masteroppgaver/004/4642/masteroppgave.pdf). 
```r
## DCC Fit
require(rmgarch)

garch11.spec = ugarchspec(
  mean.model=list(armaOrder=c(0,0)),
  variance.model=list(garchOrder=c(1,1), model="sGARCH"),
  distribution.model="norm"
)

# Specify a DCC-GARCH model
dcc.garch11.spec <- dccspec(
  uspec = multispec( replicate(2, garch11.spec)), 
  dccOrder = c(1,1), 
  distribution = "mvt" # We model residuals with a t-distribution, given its greater kurtosis, since
                       # our returns exhibited extreme values
)

# Fit the DCC using a two-step method outlined here:
#   https://daim.idi.ntnu.no/masteroppgaver/004/4642/masteroppgave.pdf
# Note that variable 'all.ts' is an xts object with the log returns of the AUD/USD spot and the VIX 
dcc.fit <- dccfit(
  dcc.garch11.spec,
  data=all.ts,
  fit.control=list(scale=TRUE)
)

# Munge the correlations for plotting and compute mean and standard deviation of correlations 
r1 <- rcor(dcc.fit, type="R")
mr1z <- mean(window(r1.z, start="2012-07-01", end="2014-07-01"))
sr1z <- sd(window(r1.z, start="2012-07-01", end="2014-07-01"))
r1.z <- zoo(r1[1,2,], order.by=time(all.ts))

# Plot the dynamic correlations 
plot(window(r1.z, start="2012-07-01", end="2014-07-01"),  
     ylab="Conditional Correlation", sub=paste("mean:", round(mr1z,3),"sd:", round(sr1z,3)),
     xlab="Date",
     xaxt="n",
     main="Time Varying Correlation of AUD/USD and VIX Index Log Returns")
axis.Date(1, at=seq(as.Date("2012-07-01", "%Y-%m-%d"), 
                    as.Date("2014-07-01", "%Y-%m-%d"), 
                    by="2 mon"), format="%b-%Y", cex.axis=0.8)
abline(h=mr1z, lty=2, lwd=1, col="blue")
abline(h=(mr1z+sr1z), lty=2, lwd=1, col="red")
abline(h=(mr1z-sr1z), lty=2, lwd=1, col="red")
```


---
###  Potential Strategy Thesis <a name="thesis"></a>
In line with the exposition above, it is clear that while there is no immediate and direct link between the equity volatility implied by the VIX and the valuation of exchange rates, particularly the US dollar, we can take advantage of some of the implications of the VIX in informing a long USD biased trading strategy. We outline the logic of a potential trading strategy, as well as simplifying assumptions, that will recover trading signals from that the VIX contains information on 30-day market expectations of volatility.   

First, we consider the valuation of the USD relative to other currencies as a measure of its strength. In times of market turmoil in which the VIX rapidly increases, dollar valuations should fall as investors look to other safer alternatives. This is seen most prominently during the 07-08 financial crisis, in the figure below,  when sharp increases in the VIX Index was immediately followed by severe depreciation in the USD exchange rate. 

<img src="img/FinCri.png">

From this, we aim to build analytics around the VIX into our strategy. Particularly, with the VIX is a measure of 30-day market expectations, we aim to use moving averages that assign more weight to more recent dates (like exponential as opposed to simple). 

Second, we consider the impact of the Australian Dollar's own valuation, relative to the US dollar. Inherent in the AUD/USD exchange rate is not only information on the US Dollar, but on its Australian counterpart as well; that the Australian Dollar can be over or undervalued relative to the US Dollar is captured in the exchange rate. We aim to control this away through using the Big Mac Index, a measure developed by the Economist that quantifies the relative over or undervaluation of a currency through the price of a Big Mac in that country. A positive Big Mac Index suggests overvaluation, and negative otherwise. 

Third, we consider the 'volatility clustering' phenomenon in our trading strategy, and attempt to control these away through the use of conditional correlations implied from a Dynamic Conditional Correlation model (an extension of multivate GARCH models, discussed briefly in the literature review). More specifically, the fit of a DCC to our time series data will yield a vector of dynamic correlations corresponding to different windows during the period studied. 

Hence, from this, the trading strategy we arrive at is the following: 






We hope to measure the performance of this improved strategy over a naive strategy in which we go long USD when the VIX is 2 standard deviations below its mean, go short when the VIX is 2 standard deviations above the mean, and hold no position otherwise. 


---
### Trading Strategy Implementation - Naive Strategy <a name="naive"></a>
We now implement the given naive trading strategy: go long USD when the VIX is 2 standard deviations below its mean, go short when the VIX is 2 standard deviations above the mean, and hold no position otherwise. 

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plot

from pyalgotrade import strategy, plotter
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma, stats
from pyalgotrade.stratanalyzer import sharpe, drawdown, returns

from QuProj import naiveStrat2, printMetrics
naive = naiveStrat2(15, "VixDatFin2.csv", "AudDatFin2.csv")
printMetrics(naive)


Final Value: $10101.50 // Cumulative Return: 1.02% // Sharpe: -0.71  // Max Drawdown: 2.12%


The strategy posts a meager positive return; the negative Sharpe ratio, although to be taken with caution, is discouraging. Before seeing if we can do better, we also consider its performance across narrower, rolling 2-year windows in our training set. As mentioned earlier, we run the strategy in 46 486-day long windows, from which we derive summary statistics around the strategy's performance analytics.  

In [ ]:
from QuProj import 

---
### Trading Strategy Implementation - Improved Strategy <a name="improved"></a>




---
### Results from Out of Sample Validation Backtests <a name="backtest"></a>
To backtest the strategies outlined in this notebook properly out of sample, we use data from June 27, 2014 to June June 29, 2018. For each strategy, we run two backtests. First, we implement the strategy over the entire period (730 days or around 3 years) recording relevant statistics and technical metrics, considering the strategy on a longer term horizon. Second, we implement the strategy again over the same period, but instead, on 45 rolling windows of 486 days (approx. 2 years). From this, we present summary statistics from the 45 different set of relevant metrics we record for the implementation in each window. Then, we repeat this for GBP/USD and EUR/USD over the entire 3 year period. 



---
### Discussion and Further Research <a name="disc"></a>


### Bibliography<a name="bib"></a>